# Class 23: Classification

Plan for today:
- Reivew/continuation of cross-validation
- Other classifiers
- Building a kNN classifier
- Features normalization


In [1]:
import YData

# YData.download.download_class_code(23)   # get class code    
# YData.download.download_class_code(23, TRUE) # get the code with the answers 

# YData.download.download_homework(9)  # downloads the homework 

# project review template
# YData.download.download_class_file('reviewer_template.ipynb', 'homework')

In [2]:
## If you are using Google Colabs, you should install the YData packages and mount the your google drive by uncommenting and running the code below.

# !pip install https://github.com/emeyers/YData_package/tarball/master
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
import statistics
import pandas as pd
import numpy as np
import seaborn as sns
import plotly.express as px
from urllib.request import urlopen

import matplotlib.pyplot as plt
%matplotlib inline

## 1. Intro to Machine Learning:  Features (X) and labels (y)

In supervised machine learning, we use a computer algorithm called a "pattern classifier" to learn relationships between a set of features X, and a label y. When the classifier is given new examples X, it can then make new predictions y. 


In [4]:
penguins = sns.load_dataset("penguins")

penguins = penguins.dropna()

penguins = penguins.sample(frac = 1)

penguins.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
5,Adelie,Torgersen,39.3,20.6,190.0,3650.0,Male
268,Gentoo,Biscoe,44.9,13.3,213.0,5100.0,Female
131,Adelie,Torgersen,43.1,19.2,197.0,3500.0,Male
40,Adelie,Dream,36.5,18.0,182.0,3150.0,Female
191,Chinstrap,Dream,53.5,19.9,205.0,4500.0,Male


To begin the classification process, let's store the features (X) and the labels (y) in separate names called `X_penguin_features` and `y_penguin_labels` respectively. 

In [5]:
# get the features and the labels

X_penguin_features = penguins[['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g']]

y_penguin_labels = penguins['species']


## 2. Cross-validation

To avoid over-fitting, we need to split our data into a training and test set. 

The classifier "learns" the relationship between features (X) and labels (y) on the **training set**.

The classifier makes predictions on the features (X) of the **test set**. 

We compare the classifier's predictions on the test features (X) to the actual labels y, to get a more accuracy assessment of the **classification accuracy**.


Let's try this now...



We can also use the scikit-learn `train_test_split()` function to generate training and test splits of our data 

In [6]:
# manually create a training with 250 examples, and a test set that has the rest of the data

X_train_manual = X_penguin_features.iloc[0:250, :]
y_train_manual = y_penguin_labels.iloc[0:250]

X_test_manual = X_penguin_features.iloc[250:, :]
y_test_manual = y_penguin_labels.iloc[250:]


print(X_train_manual.shape)
print(X_test_manual.shape)


(250, 4)
(83, 4)


In [7]:
from sklearn.model_selection import train_test_split

# split data into a training and test set

X_train, X_test, y_train, y_test = train_test_split(X_penguin_features,  
                                                    y_penguin_labels, 
                                                    random_state = 0)

print(X_train.shape)
print(X_test.shape)

X_train.head(3)


(249, 4)
(84, 4)


,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
266,45.5,13.9,210.0,4200.0
89,38.9,18.8,190.0,3600.0
230,40.9,13.7,214.0,4650.0


In [8]:
from sklearn.neighbors import KNeighborsClassifier


# construct a classifier
knn = KNeighborsClassifier(n_neighbors = 1) 

# “train” the classifier (which for a KNN classifier just involves memorizing the training data)
knn.fit(X_train, y_train) 



,n_neighbors,1
,weights,'uniform'
,algorithm,'auto'
,leaf_size,30
,p,2
,metric,'minkowski'
,metric_params,None
,n_jobs,None


In [9]:
# get the predictions

penguin_preditions = knn.predict(X_test)

penguin_preditions

array(['Gentoo', 'Chinstrap', 'Adelie', 'Adelie', 'Adelie', 'Adelie',
       'Chinstrap', 'Adelie', 'Adelie', 'Adelie', 'Adelie', 'Gentoo',
       'Chinstrap', 'Adelie', 'Gentoo', 'Adelie', 'Gentoo', 'Adelie',
       'Adelie', 'Gentoo', 'Adelie', 'Adelie', 'Adelie', 'Gentoo',
       'Chinstrap', 'Adelie', 'Adelie', 'Gentoo', 'Gentoo', 'Gentoo',
       'Chinstrap', 'Gentoo', 'Gentoo', 'Gentoo', 'Chinstrap', 'Gentoo',
       'Gentoo', 'Adelie', 'Chinstrap', 'Gentoo', 'Gentoo', 'Adelie',
       'Chinstrap', 'Adelie', 'Adelie', 'Adelie', 'Chinstrap', 'Adelie',
       'Gentoo', 'Chinstrap', 'Adelie', 'Adelie', 'Adelie', 'Adelie',
       'Adelie', 'Adelie', 'Adelie', 'Gentoo', 'Adelie', 'Gentoo',
       'Adelie', 'Adelie', 'Adelie', 'Adelie', 'Adelie', 'Chinstrap',
       'Gentoo', 'Gentoo', 'Adelie', 'Gentoo', 'Chinstrap', 'Adelie',
       'Adelie', 'Chinstrap', 'Adelie', 'Gentoo', 'Gentoo', 'Gentoo',
       'Adelie', 'Chinstrap', 'Gentoo', 'Adelie', 'Chinstrap', 'Adelie'],
      dtype=obje

In [10]:
# Get the prediction accuracy 

np.mean(penguin_preditions == y_test)



np.float64(0.8095238095238095)

In [11]:
# Test the classifier on the test set using the .score() method

knn.score(X_test, y_test) # prediction accuracy on the test set

0.8095238095238095

In [12]:
# What happens if we test the classifier on the training set? 

knn.score(X_train, y_train) # prediction accuracy on the training set



1.0

### K-fold cross-validation

In k-fold cross-validation we split our data into k-parts (note, the k here has no relation to the k in k-Nearest Neighbor - it is just that k is a frequent letter to use in math to denote integer values).  

To run a k-fold cross-validation analysis, we train the classifier on k-1 parts of the data and test it on the remaining part. We repeat this process k times to get k classification accuracies. We then take the average of these results as our estimate of our overall classification accuracy. 

We can use the scikit-learn `cross_val_score()` to easily do this...


In [13]:
from sklearn.model_selection import cross_val_score

knn = KNeighborsClassifier(n_neighbors = 1) # construct knn classifier

# do 5-fold cross-validation
scores = cross_val_score(knn, X_penguin_features,  y_penguin_labels, cv = 5)

print(scores)

print(scores.mean())

[0.76119403 0.88059701 0.88059701 0.84848485 0.84848485]
0.8438715513342379


## 3. Other classifiers

Many other types of classifiers that have been created. Scikit-learn makes it very easy to try out a range of classifiers. 

Let's explore the Support Vector Machine, and Random Forest Classifier on our penguin data...


In [14]:
# Suppress ConvergenceWarning - please ignore this code 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.filterwarnings("ignore", category=ConvergenceWarning)
warnings.filterwarnings("ignore", category=FutureWarning)


# Try a support vector machine (SVM)

from sklearn.svm import LinearSVC

svm = LinearSVC()   # max_iter=10000

scores = cross_val_score(svm, X_penguin_features,  y_penguin_labels, cv = 5)

print(scores.mean())


0.9909995477159657


In [15]:
# Try a random forest

from sklearn.ensemble import RandomForestClassifier

random_forest = RandomForestClassifier()

scores = cross_val_score(random_forest, X_penguin_features,  y_penguin_labels, cv = 5)

print(scores.mean())

0.9730438715513342


## 4. Building the KNN classifier

So far we have used the KNN classifier (and a few other classifiers). Let's now see if we can write code that will implement the KNN classifier.

We will do this by writing a several helper functions that build on each other. These functions are: 

1. `euclid_dist(x1, x2)`: finds the Euclidean distance between two points `x1` and `x2`

2. `get_labels_and_distances(test_point, X_train_features, y_train_labels)`: This function finds the distance between a test point and all the training points. It returns a DataFrame with the distance from all training points and the training labels for each point.

3. `classify_point(test_point, k, X_train_features, y_train_labels)`: Classifies which class a test point belongs to

4. `classify_all_test_data(X_test_data, k, X_train_features, y_train_labels)`: Classifiers which class all test points below to.


Let's start by writing a function that can get the Euclidean distance between two points `x` and `z`: 

$$dist(x, z) = \sqrt{\Sigma_{i = 1}^d (x_i - z_i)^2)}$$


In [16]:
def euclid_dist(x1, x2):
    return np.sqrt(np.sum((x1 - x2)**2))


# test our function 
my_vec1 = np.array([1, 2, 3, 4])
my_vec2 = np.array([2, 3, 4, 5])

euclid_dist(my_vec1, my_vec2)

np.float64(2.0)

In [17]:
# Let's now write a function that returns the labels and distances 
# between a training point and all the test points


def get_labels_and_distances(test_point, X_train_features, y_train_labels):
    
    the_distances = []
    
    # get the distance between the test point and all training points
    for i in range(X_train_features.shape[0]):
        the_distances.append(euclid_dist(test_point, X_train_features.iloc[i]))

    
    # Create a DataFrame with the training labels and distances 
    labels_and_distances = pd.DataFrame({'label': y_train_labels, 'distance':the_distances})
    return labels_and_distances


test_data_point = X_test.iloc[0]
test_label = y_test.iloc[0]

labels_and_distances = get_labels_and_distances(test_data_point, X_train, y_train)

labels_and_distances.head(5)

,label,distance
266,Gentoo,800.077934
89,Adelie,1400.366913
230,Gentoo,350.118351
143,Adelie,1275.390313
2,Adelie,1750.205408


In [18]:
# get the k closest neighbors

k = 5

sorted_labels_dist = labels_and_distances.sort_values("distance")

sorted_labels_dist = sorted_labels_dist.iloc[0:k]

sorted_labels_dist

,label,distance
277,Gentoo,1.473092
273,Gentoo,5.481788
241,Gentoo,6.236986
244,Gentoo,7.382412
322,Gentoo,25.722558


In [19]:
# get the majority label

count_table = sorted_labels_dist.groupby("label").count().reset_index()

sorted_count_table = count_table.sort_values("distance", ascending = False)

sorted_count_table.iloc[0]["label"]

'Gentoo'

In [20]:
# write a function to do the classification on a test point 
# by putting together all the pieces

def classify_point(test_point, k, X_train_features, y_train_labels):
    
    labels_and_distances =  get_labels_and_distances(test_point, 
                                                     X_train_features, 
                                                     y_train_labels)

    sorted_labels_dist = labels_and_distances.sort_values("distance")
    sorted_labels_dist = sorted_labels_dist.iloc[0:k]
    
    
    count_table = sorted_labels_dist.groupby("label").count().reset_index()
    sorted_count_table = count_table.sort_values("distance", ascending = False)
    majority_class = sorted_count_table.iloc[0]["label"]
    
    return majority_class



prediction = classify_point(test_data_point, 5, X_train, y_train)

print(prediction)

print(test_label)

Gentoo
Gentoo


In [21]:
# classify a full test set

def classify_all_test_data(X_test_data, k, X_train_features, y_train_labels):
    
    predictions = []
    
    for i in range(X_test_data.shape[0]):
        
        curr_test_point = X_test_data.iloc[i]
        
        curr_prediction = classify_point(curr_test_point, 
                                         k, 
                                         X_train_features, 
                                         y_train_labels)
        
        predictions.append(curr_prediction)

    return np.array(predictions)
    
    
all_predictions = classify_all_test_data(X_test, 5, X_train, y_train)

all_predictions


array(['Gentoo', 'Adelie', 'Adelie', 'Adelie', 'Adelie', 'Adelie',
       'Gentoo', 'Adelie', 'Adelie', 'Adelie', 'Adelie', 'Gentoo',
       'Adelie', 'Adelie', 'Gentoo', 'Adelie', 'Gentoo', 'Adelie',
       'Adelie', 'Gentoo', 'Adelie', 'Adelie', 'Chinstrap', 'Gentoo',
       'Adelie', 'Adelie', 'Adelie', 'Gentoo', 'Gentoo', 'Adelie',
       'Adelie', 'Gentoo', 'Gentoo', 'Gentoo', 'Adelie', 'Gentoo',
       'Gentoo', 'Adelie', 'Adelie', 'Gentoo', 'Gentoo', 'Adelie',
       'Chinstrap', 'Adelie', 'Adelie', 'Adelie', 'Adelie', 'Adelie',
       'Gentoo', 'Adelie', 'Adelie', 'Adelie', 'Adelie', 'Gentoo',
       'Adelie', 'Adelie', 'Adelie', 'Gentoo', 'Adelie', 'Gentoo',
       'Adelie', 'Adelie', 'Adelie', 'Adelie', 'Chinstrap', 'Adelie',
       'Gentoo', 'Gentoo', 'Adelie', 'Gentoo', 'Chinstrap', 'Adelie',
       'Adelie', 'Chinstrap', 'Adelie', 'Gentoo', 'Gentoo', 'Gentoo',
       'Adelie', 'Adelie', 'Gentoo', 'Adelie', 'Chinstrap', 'Adelie'],
      dtype='<U9')

In [22]:
# get the classification accuracy

np.mean(all_predictions == y_test)



np.float64(0.7857142857142857)

## 5. Feature normalization

If you look at the features we have been using in our analyses, you will notice that they are on very different scales. This is quite problematic for a KNN classifier since the classifier is finding the distance between each data point, so features that have large values will dominate this distance. 

Let's explore the scales that different features have by looking at some descriptive statistics. In particular, let's go back to the manually created `X_train`, `X_test`, `y_train`, `y_test` to examine the scale that different features are measured on.


In [23]:
# Create the training and test splots of the data using train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_penguin_features,  
                                                    y_penguin_labels, 
                                                    random_state = 0)

# Get summary statistics of the training data using the .describe() method
X_train.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,249.000000,249.000000,249.000000,249.000000
mean,44.152610,17.086345,201.457831,4239.156627
std,5.515645,2.003338,14.002276,808.743108
min,32.100000,13.100000,178.000000,2700.000000
25%,39.700000,15.400000,190.000000,3650.000000
50%,44.500000,17.300000,198.000000,4050.000000
75%,48.600000,18.700000,214.000000,4800.000000
max,59.600000,21.100000,230.000000,6300.000000


Let's do a z-score transformation of our features which set the mean of the features to 0 and the standard deviation to 1. We can do this using the using the `StandardScaler()` object as follows: 

1. Create a new `StandardScaler()` object using `scaler = StandardScaler()` 

2. Have the `scaler` object learn the means and standard deviations of our training data by calling the `scaler.fit(X)` function on the training data.

3. Use the fit `scaler` object to transform both the training and test features so that all features are on a similar scale by calling the `.transform(X)` method. 


In [24]:
from sklearn.preprocessing import StandardScaler


# learning the mean and standard deviations to scale the features

scaler = StandardScaler()

scaler.fit(X_train)


,copy,True
,with_mean,True
,with_std,True


In [25]:
# z-score transform the features 

X_train_transformed = scaler.transform(X_train)
X_test_transformed = scaler.transform(X_test)

type(X_test_transformed)

numpy.ndarray

Let's now look at our transformed training data...

In [26]:
# view descriptive statistics on the transformed features

X_train_transformed_df = pd.DataFrame(X_train_transformed, 
                                      columns = X_train.columns)

X_train_transformed_df.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,2.490000e+02,2.490000e+02,2.490000e+02,2.490000e+02
mean,3.388633e-16,-4.993774e-17,-6.741595e-16,4.788673e-16
std,1.002014e+00,1.002014e+00,1.002014e+00,1.002014e+00
min,-2.189569e+00,-1.993860e+00,-1.678661e+00,-1.906980e+00
25%,-8.088951e-01,-8.434634e-01,-8.199316e-01,-7.299515e-01
50%,6.310943e-02,1.068641e-01,-2.474452e-01,-2.343607e-01
75%,8.079466e-01,8.071055e-01,8.975276e-01,6.948720e-01
max,2.806290e+00,2.007519e+00,2.042500e+00,2.553338e+00


Let's see how our classification accuracy changes using the z-score transformed data

In [27]:
# apply KNN classification on the normalized features

knn = KNeighborsClassifier(n_neighbors = 1) 
knn.fit(X_train_transformed, y_train)
knn.score(X_test_transformed, y_test)

0.9642857142857143

In order to transform our features inside a cross-validation loop, we can set up a pipeline. This pipeline will do the following:

1. It will split the data into a training and test set
2. It will fit the transformation of the features on the training set (i.e., learn the means and standard deviations on the training set). 
3. It will apply a z-score transformation of the training and test set based on the features learned in step 2
4. It will train the classifier on the transformed data
5. It will measure the classification accuracy on the test data
6. It will repeat this process k times, where k here refers to how many cross-validation splits we are using

In order to do this in scikit-learn we can use a `Pipeline` object which sets up the stages of transformation and classification. We can then use the `cross_val_score()` function to run cross-validation on this pipeline. 

In [28]:
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import KFold


# create a pipeline for running cross-validation with feature normalization

# components that go into the pipeline
scaler = StandardScaler()
knn = KNeighborsClassifier(n_neighbors = 1) 

# build the pipeline
#pipeline = Pipeline([('transformer', scaler), ('estimator', knn)])
pipeline = make_pipeline(scaler, knn)

# get the cross-validation scores
scores = cross_val_score(pipeline, 
                         X_penguin_features, 
                         y_penguin_labels, 
                         cv = 5)


# print out the mean score over the 5 cross-validation splits
scores.mean()

np.float64(0.9850293984622344)

In [29]:
%%capture

# You can run this code to covert this Jupyter notebook into a pdf
!quarto render class_23_answers.ipynb --cache-refresh --to pdf 